In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup
import numpy as np
import requests
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [38]:
import json

# Load JSON file
with open("vetting_playlist.json", "r", encoding="utf-8") as file:
    data = json.load(file)

data

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0bw8158RM7roegphH1GYCx'},
 'followers': {'href': None, 'total': 1},
 'href': 'https://api.spotify.com/v1/playlists/0bw8158RM7roegphH1GYCx?additional_types=track',
 'id': '0bw8158RM7roegphH1GYCx',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d00001e023b7ab410202ecd5aa7a61eb1ab67616d00001e02670f91f937e105b878366cffab67616d00001e02d4d231e11ce86d2a6e6f9414ab67616d00001e02df1884a7ebca0168bdcaf69e',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d00001e023b7ab410202ecd5aa7a61eb1ab67616d00001e02670f91f937e105b878366cffab67616d00001e02d4d231e11ce86d2a6e6f9414ab67616d00001e02df1884a7ebca0168bdcaf69e',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d00001e023b7ab410202ecd5aa7a61eb1ab67616d00001e02670f91f937e105b878366cffab67616d00001e02d4d231e11ce86d2a6e6f9414ab67616d00001e02df1884a7ebca0168bdca

In [29]:
data.keys()

dict_keys(['collaborative', 'description', 'external_urls', 'followers', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

vetting_playlist

In [30]:
data['tracks'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [31]:
data['tracks']['items'][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [32]:
data['tracks']['items'][0]['track'].keys()

dict_keys(['preview_url', 'available_markets', 'explicit', 'type', 'episode', 'track', 'album', 'artists', 'disc_number', 'track_number', 'duration_ms', 'external_ids', 'external_urls', 'href', 'id', 'name', 'popularity', 'uri', 'is_local'])

In [7]:
len(data['tracks']['items'])

146

In [39]:
combo_df = {
    "playlist": [],
    "track": [],
    "popularity": []
}

In [40]:
for i in range(146):
    pop = data['tracks']['items'][i]['track']['popularity']
    track_name = data['tracks']['items'][i]['track']['name']
    combo_df['playlist'].append("Ms.Spindt")
    combo_df['popularity'].append(pop)
    combo_df['track'].append(track_name)

In [44]:
vetting = pd.DataFrame(combo_df)

In [45]:
vetting.head()

,playlist,track,popularity
0,Ms.Spindt,Dunes,0
1,Ms.Spindt,Future People,0
2,Ms.Spindt,Don't Wanna Fight,0
3,Ms.Spindt,Gimme All Your Love,0
4,Ms.Spindt,Only For You,0


In [48]:
vetting.sort_values(['popularity'], ascending=[False])

,playlist,track,popularity
126,Ms.Spindt,Yellow,88
122,Ms.Spindt,Apocalypse,87
88,Ms.Spindt,Washing Machine Heart,81
117,Ms.Spindt,Paper Planes,79
25,Ms.Spindt,Where'd All the Time Go?,76
...,...,...,...
90,Ms.Spindt,How It Ends,0
56,Ms.Spindt,You're Somebody Else,0
57,Ms.Spindt,Turn On Me,0
58,Ms.Spindt,Sunflower (feat. Steve Lacy),0


In [11]:
data['tracks']['items'][0]['track']['artists'][0].keys()

dict_keys(['external_urls', 'href', 'id', 'name', 'type', 'uri'])

In [12]:
data['tracks']['items'][0]['track']['artists'][0]['name']

'Alabama Shakes'

In [13]:
file_paths = [
    'mpd.slice.15000-15999.json',
    'mpd.slice.16000-16999.json',
    'mpd.slice.17000-17999.json',
    'mpd.slice.18000-18999.json',
    'mpd.slice.19000-19999.json',
    'mpd.slice.20000-20999.json',
    'mpd.slice.21000-21999.json',
    'mpd.slice.22000-22999.json',
    'mpd.slice.23000-23999.json',
    'mpd.slice.24000-24999.json',
    'mpd.slice.25000-25999.json',
    'mpd.slice.26000-26999.json',
    'mpd.slice.27000-27999.json',
    'mpd.slice.28000-28999.json',
    'mpd.slice.29000-29999.json',
    'mpd.slice.30000-30999.json',
]

all_data = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        for playlist in data['playlists']:
            playlist_name = playlist['name']
            for track in playlist['tracks']:
                all_data.append({
                    "playlist_name": playlist_name,
                    "position_in_playlist": track["pos"],
                    "track_id": track['track_uri'],
                    "track_name": track['track_name'],
                    "artist_name": track['artist_name'],
                    "duration_ms": track['duration_ms'],
                    "album_name": track['album_name']
                })
                # Add and remove for DBSCAN
                '''
                if len(all_data) >= 1000:
                    break
            if len(all_data) >= 1000:
                break
        if len(all_data) >= 1000:
            break
            '''
df = pd.DataFrame(all_data)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077598 entries, 0 to 1077597
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   playlist_name         1077598 non-null  object
 1   position_in_playlist  1077598 non-null  int64 
 2   track_id              1077598 non-null  object
 3   track_name            1077598 non-null  object
 4   artist_name           1077598 non-null  object
 5   duration_ms           1077598 non-null  int64 
 6   album_name            1077598 non-null  object
dtypes: int64(2), object(5)
memory usage: 57.6+ MB


In [15]:
df.head()

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP


In [16]:
playlist_track_counts = df.groupby(['playlist_name', 'track_id', 'track_name', 'artist_name']).size().reset_index(name='count')

In [17]:
def calculate_popularity_score(group):
    min_count = group['count'].min()
    max_count = group['count'].max()
    if max_count == min_count:
        group['popularity_score'] = 5  # Assign a neutral score if all counts are the same
    else:
        group['popularity_score'] = ((group['count'] - min_count) / (max_count - min_count) * 9 + 1).astype(int)
    return group

playlist_track_counts = playlist_track_counts.groupby('playlist_name').apply(calculate_popularity_score)

In [18]:
df_with_playlist_popularity = df.merge(
    playlist_track_counts[['playlist_name', 'track_id', 'popularity_score']],
    on=['playlist_name', 'track_id'],
    how='left'
)

In [23]:
specific_playlist_name = "Vetting"
sorted_tracks_in_playlist = df_with_playlist_popularity[df_with_playlist_popularity['playlist_name'] == specific_playlist_name].sort_values(by='popularity_score', ascending=False)
print(sorted_tracks_in_playlist.head())

Empty DataFrame
Columns: [playlist_name, position_in_playlist, track_id, track_name, artist_name, duration_ms, album_name, popularity_score]
Index: []


In [24]:
df_with_playlist_popularity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1077598 entries, 0 to 1077597
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   playlist_name         1077598 non-null  object
 1   position_in_playlist  1077598 non-null  int64 
 2   track_id              1077598 non-null  object
 3   track_name            1077598 non-null  object
 4   artist_name           1077598 non-null  object
 5   duration_ms           1077598 non-null  int64 
 6   album_name            1077598 non-null  object
 7   popularity_score      1077598 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 74.0+ MB


In [25]:
df_with_playlist_popularity.head(25)

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name,popularity_score
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham,5
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham,5
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons,5
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For,5
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP,5
5,Wedding Music,5,spotify:track:6bSY42jHs2rQoeAAlqxilu,Perfect for Me,Ron Pope,215600,The New England Sessions,5
6,Wedding Music,6,spotify:track:2FGb3gJbKToDII7q6Kfp4i,I'm Yours,Ron Pope,136544,Ron Pope: Live and Unplugged In New York,5
7,Wedding Music,7,spotify:track:72cM8kDitkjEk1VVSvRH4k,Last First Kiss,Ron Pope,268930,The New England Sessions,5
8,Wedding Music,8,spotify:track:5zyNza76kEYHoRgh6NF83X,Anchored In You,Shawn Mullins,198933,The Essential Shawn Mullins,5
9,Wedding Music,9,spotify:track:7L605WhF5EGf34ggj87yK6,You And Me,Lifehouse,195493,Lifehouse,5


In [22]:
df_with_playlist_popularity.loc[456789]

playlist_name                                        Chillax
position_in_playlist                                      49
track_id                spotify:track:09Dpwk2CcMJ1V9HSYIQSmQ
track_name                                          Sube Ink
artist_name                                          Michita
duration_ms                                           249808
album_name                       A Full Life - Instrumentals
popularity_score                                           1
Name: 456789, dtype: object